In [15]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from codecarbon import EmissionsTracker
from ptflops import get_model_complexity_info
import os
import pandas as pd

In [16]:

# ----------------------------
# Device Setup
# ----------------------------
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [17]:
# ----------------------------
# EfficientNet-V2-S (No Tuning)
# ----------------------------
class EfficientNetV2S_Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.base_model = models.efficientnet_v2_s(weights='IMAGENET1K_V1')
        # Replace the classifier
        in_features = self.base_model.classifier[1].in_features
        self.base_model.classifier[1] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)


[codecarbon INFO @ 23:35:55] Energy consumed for RAM : 0.000882 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 23:35:55] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 23:35:55] Energy consumed for All CPU : 0.003753 kWh
[codecarbon INFO @ 23:35:55] Energy consumed for all GPUs : 0.002690 kWh. Total GPU Power : 31.564462834192522 W
[codecarbon INFO @ 23:35:55] 0.007326 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 23:35:55] 0.008097 g.CO2eq/s mean an estimation of 255.33633315815246 kg.CO2eq/year


In [25]:
# ----------------------------
# Dataset & DataLoader
# ----------------------------
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset   = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)


[codecarbon INFO @ 23:39:29] Energy consumed for RAM : 0.001472 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 23:39:29] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 23:39:29] Energy consumed for All CPU : 0.006262 kWh
[codecarbon INFO @ 23:39:29] Energy consumed for all GPUs : 0.004586 kWh. Total GPU Power : 32.36457010676542 W
[codecarbon INFO @ 23:39:29] 0.012320 kWh of electricity and 0.000000 L of water were used since the beginning.


In [26]:
# ----------------------------
# Train & Validation Functions
# ----------------------------
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
    return total_loss / len(loader.dataset)

def validate(model, loader, criterion):
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    acc = 100 * correct / total
    return val_loss / total, acc

[codecarbon INFO @ 23:39:43] Energy consumed for RAM : 0.001510 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 23:39:43] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 23:39:43] Energy consumed for All CPU : 0.006427 kWh
[codecarbon INFO @ 23:39:43] Energy consumed for all GPUs : 0.004712 kWh. Total GPU Power : 32.62619758864525 W
[codecarbon INFO @ 23:39:43] 0.012649 kWh of electricity and 0.000000 L of water were used since the beginning.


In [27]:
# ----------------------------
# Model Setup
# ----------------------------
num_classes = 10
model = EfficientNetV2S_Model(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ----------------------------
# FLOPs Calculation
# ----------------------------
macs, params = get_model_complexity_info(model, (3, 64, 64),
                                         as_strings=True,
                                         print_per_layer_stat=False,
                                         verbose=False)
macs_value = macs.replace('Mac', '').strip()
if 'G' in macs_value:
    macs_num = float(macs_value.replace('G', '')) * 2
    flops_per_forward = f"{macs_num:.2f} GFLOPs"
elif 'M' in macs_value:
    macs_num = float(macs_value.replace('M', '')) / 1000 * 2
    flops_per_forward = f"{macs_num:.3f} GFLOPs"
else:
    flops_per_forward = f"{macs_value} MACs"

print(f"FLOPs per forward pass: {flops_per_forward} | Params: {params}")

[codecarbon INFO @ 23:39:53] Energy consumed for RAM : 0.001538 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 23:39:53] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 23:39:53] Energy consumed for All CPU : 0.006544 kWh
[codecarbon INFO @ 23:39:53] Energy consumed for all GPUs : 0.004803 kWh. Total GPU Power : 32.642724279121815 W
[codecarbon INFO @ 23:39:53] 0.012885 kWh of electricity and 0.000000 L of water were used since the beginning.


FLOPs per forward pass: 0.125 GFLOPs | Params: 20.19 M


In [28]:
# ----------------------------
# Energy & CO₂ Tracking
# ----------------------------
output_dir = "./emissions_efficientnet_no_tuning"
os.makedirs(output_dir, exist_ok=True)

tracker = EmissionsTracker(
    project_name="EfficientNetV2S_CIFAR10_NoTuning",
    output_dir=output_dir,
    measure_power_secs=1
)
tracker.start()
start_time = time.time()

[codecarbon WARNING @ 23:40:07] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 23:40:07] [setup] RAM Tracking...
[codecarbon INFO @ 23:40:07] [setup] CPU Tracking...
[codecarbon INFO @ 23:40:07] Energy consumed for RAM : 0.001576 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 23:40:07] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 23:40:07] Energy consumed for All CPU : 0.006708 kWh
[codecarbon INFO @ 23:40:07] Energy consumed for all GPUs : 0.004928 kWh. Total GPU Power : 32.61647121039733 W
[codecarbon INFO @ 23:40:07] 0.013213 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon WARNING @ 23:40:08] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 23:40:08] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/po

In [29]:
# ----------------------------
# Training Loop (4 Epochs)
# ----------------------------
for epoch in range(4):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)
    print(f"Epoch {epoch+1}/4 | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.2f}%")


Streaming output truncated to the last 5000 lines.
[codecarbon INFO @ 23:43:18] Energy consumed for All CPU : 0.002217 kWh
[codecarbon INFO @ 23:43:18] Energy consumed for all GPUs : 0.003458 kWh. Total GPU Power : 69.56670991365182 W
[codecarbon INFO @ 23:43:18] 0.006196 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 23:43:18] Energy consumed for RAM : 0.002100 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 23:43:18] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 23:43:18] Energy consumed for All CPU : 0.008937 kWh
[codecarbon INFO @ 23:43:18] Energy consumed for all GPUs : 0.008395 kWh. Total GPU Power : 70.11009515201508 W
[codecarbon INFO @ 23:43:18] 0.019431 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 23:43:19] Energy consumed for RAM : 0.000523 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 23:43:19] Delta energy consumed for CPU with constant : 0.

Epoch 1/4 | Train Loss: 0.4322 | Val Acc: 90.40%


Streaming output truncated to the last 5000 lines.
[codecarbon INFO @ 23:54:18] Energy consumed for All CPU : 0.009898 kWh
[codecarbon INFO @ 23:54:18] Energy consumed for all GPUs : 0.016023 kWh. Total GPU Power : 70.7039070539253 W
[codecarbon INFO @ 23:54:18] 0.028246 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 23:54:18] Energy consumed for RAM : 0.003903 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 23:54:18] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 23:54:18] Energy consumed for All CPU : 0.016615 kWh
[codecarbon INFO @ 23:54:18] Energy consumed for all GPUs : 0.020962 kWh. Total GPU Power : 70.86188267873256 W
[codecarbon INFO @ 23:54:18] 0.041479 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 23:54:19] Energy consumed for RAM : 0.002327 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 23:54:19] Delta energy consumed for CPU with constant : 0.0

Epoch 2/4 | Train Loss: 0.2411 | Val Acc: 91.19%


Streaming output truncated to the last 5000 lines.
[codecarbon INFO @ 00:05:19] Energy consumed for All CPU : 0.017605 kWh
[codecarbon INFO @ 00:05:19] Energy consumed for all GPUs : 0.028620 kWh. Total GPU Power : 70.47670868412506 W
[codecarbon INFO @ 00:05:19] 0.050359 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 00:05:19] Energy consumed for RAM : 0.005712 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 00:05:19] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 00:05:19] Energy consumed for All CPU : 0.024320 kWh
[codecarbon INFO @ 00:05:19] Energy consumed for all GPUs : 0.033558 kWh. Total GPU Power : 63.37749279783633 W
[codecarbon INFO @ 00:05:19] 0.063591 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 00:05:20] Energy consumed for RAM : 0.004137 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 00:05:20] Delta energy consumed for CPU with constant : 0.

Epoch 3/4 | Train Loss: 0.1881 | Val Acc: 93.26%


Streaming output truncated to the last 5000 lines.
[codecarbon INFO @ 00:16:21] Energy consumed for All CPU : 0.025323 kWh
[codecarbon INFO @ 00:16:21] Energy consumed for all GPUs : 0.041228 kWh. Total GPU Power : 62.360864128719996 W
[codecarbon INFO @ 00:16:21] 0.072498 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 00:16:21] Energy consumed for RAM : 0.007526 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 00:16:21] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 00:16:21] Energy consumed for All CPU : 0.032042 kWh
[codecarbon INFO @ 00:16:21] Energy consumed for all GPUs : 0.046167 kWh. Total GPU Power : 67.81472973139927 W
[codecarbon INFO @ 00:16:21] 0.085735 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 00:16:22] Energy consumed for RAM : 0.005950 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 00:16:22] Delta energy consumed for CPU with constant : 0

Epoch 4/4 | Train Loss: 0.1517 | Val Acc: 91.96%


In [30]:
# ----------------------------
# Stop Tracking & Log Results
# ----------------------------
emissions = tracker.stop()
elapsed = time.time() - start_time

[codecarbon INFO @ 00:25:48] Energy consumed for RAM : 0.007501 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 00:25:48] Delta energy consumed for CPU with constant : 0.000005 kWh, power : 42.5 W
[codecarbon INFO @ 00:25:48] Energy consumed for All CPU : 0.031936 kWh
[codecarbon INFO @ 00:25:48] Energy consumed for all GPUs : 0.051217 kWh. Total GPU Power : 36.40945956512251 W
[codecarbon INFO @ 00:25:48] 0.090654 kWh of electricity and 0.000000 L of water were used since the beginning.


In [32]:
# ----------------------------
# Stop Tracking & Show Results
# ----------------------------
emissions = tracker.stop()
elapsed = time.time() - start_time

csv_file = os.path.join(output_dir, "emissions.csv")
df = pd.read_csv(csv_file)
energy = df['energy_consumed'].iloc[-1]

print("\n==============================")
print("✅ Final Results Summary")
print("==============================")
print(f"CO₂ Emissions : {emissions:.6f} kg")
print(f"Energy Used   : {energy:.6f} kWh")
print(f"Time Taken    : {elapsed/60:.2f} min")
print(f"FLOPs         : {flops_per_forward}")
print("==============================")


[codecarbon WARNING @ 00:27:25] Tracker already stopped !
[codecarbon INFO @ 00:27:25] Energy consumed for RAM : 0.007770 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 00:27:25] Delta energy consumed for CPU with constant : 0.001143 kWh, power : 42.5 W
[codecarbon INFO @ 00:27:25] Energy consumed for All CPU : 0.033079 kWh
[codecarbon INFO @ 00:27:25] Energy consumed for all GPUs : 0.052106 kWh. Total GPU Power : 33.04927405748084 W
[codecarbon INFO @ 00:27:25] 0.092955 kWh of electricity and 0.000000 L of water were used since the beginning.



✅ Final Results Summary
CO₂ Emissions : 0.032462 kg
Energy Used   : 0.092955 kWh
Time Taken    : 47.28 min
FLOPs         : 0.125 GFLOPs
